# Lab 3.6.2: Building ReAct Agents

**Module:** 3.6 - AI Agents & Agentic Systems  
**Time:** 2 hours  
**Difficulty:** ⭐⭐⭐⭐ (Advanced Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand the ReAct (Reasoning + Acting) pattern
- [ ] Build agents that reason about which tools to use
- [ ] Implement the Think → Act → Observe loop
- [ ] Handle agent errors and iterations gracefully
- [ ] Create conversational agents with memory

---

## 📚 Prerequisites

- Completed: Lab 3.6.1 (Custom Tools)
- Knowledge of: LangChain basics, Python decorators
- Running: Ollama with `qwen3:8b` or `qwen3:32b` model

**⚠️ Important Note about Tool Calling Models:**
- Use `qwen3:8b` or `qwen3:32b` for tool calling (recommended)
- `deepseek-r1` does NOT support tool calling - it's for reasoning only
- For ReAct agents, qwen3 models provide the best balance of reasoning and tool use

---

## 🌍 Real-World Context

**The Problem:** Traditional chatbots just generate text. They can't:
- Look up real information
- Perform calculations
- Execute code
- Access external systems

**ReAct Agents solve this** by combining:
- **Re**asoning: Thinking about what to do
- **Act**ing: Using tools to get real results

**Real-World Applications:**
- **Customer Service**: Look up orders, process refunds, answer FAQs
- **Research Assistants**: Search papers, analyze data, summarize findings
- **DevOps Bots**: Check system status, run diagnostics, deploy fixes
- **Personal Assistants**: Manage calendars, send emails, book travel

---

## 🧒 ELI5: What is the ReAct Pattern?

> **Imagine you're solving a mystery...** 🔍
>
> You don't just guess the answer. You:
> 1. **THINK**: "I need to find out where the suspect was at 8 PM"
> 2. **ACT**: Check the security camera footage
> 3. **OBSERVE**: "The suspect was at the library"
> 4. **THINK**: "Now I need to verify their alibi..."
> 5. **ACT**: Call the librarian
> 6. ...and so on until you solve the mystery!
>
> **ReAct agents work the same way:**
> - **Thought**: "I need to calculate the tip amount"
> - **Action**: Use the calculator tool with "127.50 * 0.15"
> - **Observation**: "19.125"
> - **Thought**: "Now I have the answer!"
> - **Final Answer**: "The 15% tip is $19.13"

```
┌─────────────────────────────────────────────────────────────────┐
│                     The ReAct Loop                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌─────────┐                                                   │
│   │  USER   │ "What's 15% tip on $127.50?"                     │
│   └────┬────┘                                                   │
│        │                                                        │
│        ▼                                                        │
│   ┌─────────────────────────────────────────────┐               │
│   │              THOUGHT (Reasoning)            │               │
│   │  "I need to calculate 15% of 127.50.        │               │
│   │   I should use the calculator tool."        │               │
│   └─────────────────────┬───────────────────────┘               │
│                         │                                       │
│                         ▼                                       │
│   ┌─────────────────────────────────────────────┐               │
│   │               ACTION (Acting)               │               │
│   │  Tool: calculate                            │               │
│   │  Input: "127.50 * 0.15"                     │               │
│   └─────────────────────┬───────────────────────┘               │
│                         │                                       │
│                         ▼                                       │
│   ┌─────────────────────────────────────────────┐               │
│   │            OBSERVATION (Result)             │               │
│   │  "127.50 * 0.15 = 19.125"                   │               │
│   └─────────────────────┬───────────────────────┘               │
│                         │                                       │
│              ┌──────────┴──────────┐                            │
│              │  Need more steps?   │                            │
│              └──────────┬──────────┘                            │
│                   No ───┴─── Yes → Loop back to THOUGHT         │
│                         │                                       │
│                         ▼                                       │
│   ┌─────────────────────────────────────────────┐               │
│   │              FINAL ANSWER                   │               │
│   │  "The 15% tip on $127.50 is $19.13"        │               │
│   └─────────────────────────────────────────────┘               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Environment Setup

In [ ]:
# Standard imports
import os
import sys
from pathlib import Path
from typing import Optional, Dict, Any, List
import json
import time
import ast
import operator
import math
import re

# Add scripts to path
sys.path.insert(0, str(Path.cwd().parent / 'scripts'))

print("✅ Standard imports successful!")

In [ ]:
# LangChain imports
from langchain.tools import tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import StdOutCallbackHandler

print("✅ LangChain imports successful!")

In [ ]:
# Initialize the LLM
# Use qwen3:8b for faster responses during development
# Use qwen3:32b for production quality (requires more memory)
# NOTE: Do NOT use deepseek-r1 for agents - it doesn't support tool calling!

llm = Ollama(
    model="qwen3:8b",  # Change to "qwen3:32b" for better reasoning
    temperature=0.1,      # Low temperature for more consistent tool use
    base_url="http://localhost:11434",
    # request_timeout=120.0,  # 2 minute timeout for complex tasks
)

# Quick test
response = llm.invoke("Say 'Hello, I am ready!' in exactly those words.")
print(f"✅ LLM initialized!")
print(f"   Response: {response[:50]}...")

---

## Part 2: Building Our Tool Set

Let's create the tools our ReAct agent will use. These are similar to Lab 3.6.1 but optimized for agent use.

In [ ]:
# Import the SafeCalculator from our scripts
# If not available, we'll define it inline

class SafeCalculator:
    """Safe mathematical expression evaluator using AST."""
    
    OPERATORS = {
        ast.Add: operator.add,
        ast.Sub: operator.sub,
        ast.Mult: operator.mul,
        ast.Div: operator.truediv,
        ast.FloorDiv: operator.floordiv,
        ast.Mod: operator.mod,
        ast.Pow: operator.pow,
        ast.USub: operator.neg,
    }
    
    FUNCTIONS = {
        'sin': math.sin, 'cos': math.cos, 'tan': math.tan,
        'sqrt': math.sqrt, 'log': math.log, 'log10': math.log10,
        'exp': math.exp, 'abs': abs, 'round': round,
        'floor': math.floor, 'ceil': math.ceil,
    }
    
    CONSTANTS = {'pi': math.pi, 'e': math.e}
    
    def evaluate(self, expression: str) -> float:
        tree = ast.parse(expression.strip(), mode='eval')
        return self._eval_node(tree.body)
    
    def _eval_node(self, node) -> float:
        if isinstance(node, ast.Constant):
            return node.value
        elif isinstance(node, ast.Name):
            if node.id in self.CONSTANTS:
                return self.CONSTANTS[node.id]
            raise ValueError(f"Unknown constant: {node.id}")
        elif isinstance(node, ast.BinOp):
            left = self._eval_node(node.left)
            right = self._eval_node(node.right)
            return self.OPERATORS[type(node.op)](left, right)
        elif isinstance(node, ast.UnaryOp):
            return self.OPERATORS[type(node.op)](self._eval_node(node.operand))
        elif isinstance(node, ast.Call):
            func_name = node.func.id if isinstance(node.func, ast.Name) else None
            if func_name not in self.FUNCTIONS:
                raise ValueError(f"Unknown function: {func_name}")
            args = [self._eval_node(arg) for arg in node.args]
            return self.FUNCTIONS[func_name](*args)
        else:
            raise ValueError(f"Unsupported: {type(node).__name__}")

print("✅ SafeCalculator defined!")

In [ ]:
# Define our agent tools

@tool
def calculate(expression: str) -> str:
    """
    Calculate a mathematical expression. Use for any math operations.
    
    Supports: +, -, *, /, **, sqrt(), sin(), cos(), log(), etc.
    Constants: pi, e
    
    Args:
        expression: Math expression like "2 + 2" or "sqrt(16) * 3"
    
    Examples:
        calculate("127.50 * 0.15") -> "19.125"
        calculate("sqrt(144)") -> "12"
    """
    try:
        result = SafeCalculator().evaluate(expression)
        if isinstance(result, float) and result.is_integer():
            return str(int(result))
        return f"{result:.10g}"
    except Exception as e:
        return f"Error: {e}"


@tool
def search_knowledge(query: str) -> str:
    """
    Search for information in the knowledge base.
    
    Use this when you need factual information about topics like:
    - DGX Spark specifications
    - AI/ML concepts
    - Technology information
    
    Args:
        query: What to search for
    """
    # Simulated knowledge base
    knowledge = {
        "dgx spark memory": "The DGX Spark has 128GB of unified LPDDR5X memory shared between CPU and GPU.",
        "dgx spark gpu": "The DGX Spark uses the NVIDIA Blackwell GB10 Superchip with 6,144 CUDA cores and 192 Tensor Cores.",
        "dgx spark compute": "The DGX Spark delivers 1 PFLOP of FP4 compute, ~209 TFLOPS FP8, and ~100 TFLOPS BF16.",
        "react pattern": "ReAct (Reasoning + Acting) is a prompting technique that interleaves reasoning traces with actions.",
        "langchain": "LangChain is a framework for building applications with LLMs through composable components.",
        "langgraph": "LangGraph is a library for building stateful, multi-actor applications using graph-based orchestration.",
        "llama 3": "Llama 3.1 is Meta's open-source LLM available in 8B, 70B, and 405B parameter versions.",
        "transformer": "Transformers are neural network architectures using self-attention, introduced in 'Attention is All You Need'.",
    }
    
    query_lower = query.lower()
    for key, value in knowledge.items():
        if any(word in query_lower for word in key.split()):
            return value
    
    return f"No information found for: {query}. Try a different search term."


@tool
def get_current_time() -> str:
    """
    Get the current date and time.
    
    Use this when the user asks about today's date or current time.
    """
    from datetime import datetime
    now = datetime.now()
    return now.strftime("%A, %B %d, %Y at %I:%M %p")


@tool  
def analyze_text(text: str) -> str:
    """
    Analyze text and return statistics.
    
    Use this to get word count, character count, and reading time.
    
    Args:
        text: The text to analyze
    """
    words = text.split()
    chars = len(text)
    chars_no_space = len(text.replace(' ', ''))
    sentences = len(re.findall(r'[.!?]+', text))
    reading_time = len(words) / 200  # 200 WPM average
    
    return f"""Text Analysis:
- Words: {len(words)}
- Characters: {chars} (without spaces: {chars_no_space})
- Sentences: {sentences}
- Reading time: {reading_time:.1f} minutes"""


# Collect all tools
tools = [calculate, search_knowledge, get_current_time, analyze_text]

print("✅ Tools defined!")
print(f"   Available: {[t.name for t in tools]}")

---

## Part 3: The ReAct Prompt Template

The ReAct prompt is crucial - it tells the LLM how to format its reasoning and tool calls.

In [ ]:
# The standard ReAct prompt template
REACT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

prompt = PromptTemplate.from_template(REACT_PROMPT)

print("✅ ReAct prompt template created!")
print("\n📝 The prompt format:")
print("   Thought: [LLM reasoning]")
print("   Action: [tool name]")
print("   Action Input: [tool input]")
print("   Observation: [tool result]")
print("   ... (repeat as needed)")
print("   Final Answer: [response to user]")

---

## Part 4: Creating the ReAct Agent

In [ ]:
# Create the ReAct agent
agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

# Create the agent executor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,           # Show reasoning steps
    max_iterations=5,       # Prevent infinite loops
    handle_parsing_errors=True,  # Handle malformed responses
    return_intermediate_steps=True,  # Get tool call history
)

print("✅ ReAct agent created!")
print(f"   Max iterations: 5")
print(f"   Tools: {[t.name for t in tools]}")

---

## Part 5: Testing the Agent

Let's test our agent with various types of questions.

In [ ]:
# Test 1: Simple calculation
print("="*60)
print("🧪 TEST 1: Calculation")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "What is 15% tip on a $127.50 restaurant bill?"
})

print("\n" + "="*60)
print(f"📤 Final Answer: {result['output']}")
print("="*60)

In [ ]:
# Test 2: Knowledge search
print("="*60)
print("🧪 TEST 2: Knowledge Search")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "How much memory does the DGX Spark have?"
})

print("\n" + "="*60)
print(f"📤 Final Answer: {result['output']}")
print("="*60)

In [ ]:
# Test 3: Multi-step reasoning
print("="*60)
print("🧪 TEST 3: Multi-Step Reasoning")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "If DGX Spark has 128GB memory and a 70B parameter model uses about 140GB in FP16, can I run it in FP8 (which halves memory usage)?"
})

print("\n" + "="*60)
print(f"📤 Final Answer: {result['output']}")
print("="*60)

In [ ]:
# Test 4: Current time
print("="*60)
print("🧪 TEST 4: Current Time")
print("="*60 + "\n")

result = agent_executor.invoke({
    "input": "What is today's date and time?"
})

print("\n" + "="*60)
print(f"📤 Final Answer: {result['output']}")
print("="*60)

### 🔍 Examining the Agent's Reasoning

Let's look at the intermediate steps to understand how the agent thinks.

In [ ]:
# Run a query and examine intermediate steps
result = agent_executor.invoke({
    "input": "Calculate the square root of 256, then multiply by 2"
})

print("\n" + "="*60)
print("🔬 INTERMEDIATE STEPS ANALYSIS")
print("="*60)

for i, step in enumerate(result.get('intermediate_steps', [])):
    action, observation = step
    print(f"\nStep {i+1}:")
    print(f"  Tool: {action.tool}")
    print(f"  Input: {action.tool_input}")
    print(f"  Result: {observation}")

print(f"\n📤 Final Answer: {result['output']}")

---

## Part 6: Building a Custom ReAct Agent from Scratch

Let's understand the ReAct pattern better by building a simple version ourselves.

In [ ]:
class SimpleReActAgent:
    """
    A simple ReAct agent implementation from scratch.
    
    This helps you understand what's happening under the hood
    when you use LangChain's create_react_agent.
    """
    
    def __init__(self, llm, tools: List, max_iterations: int = 5):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.max_iterations = max_iterations
        self.history = []
    
    def _get_tool_descriptions(self) -> str:
        """Format tool descriptions for the prompt."""
        descriptions = []
        for name, tool in self.tools.items():
            desc = tool.description.split('\n')[0]  # First line only
            descriptions.append(f"- {name}: {desc}")
        return "\n".join(descriptions)
    
    def _parse_response(self, response: str) -> dict:
        """Parse the LLM response for thoughts, actions, and final answers."""
        result = {
            "thought": None,
            "action": None,
            "action_input": None,
            "final_answer": None
        }
        
        # Extract Thought
        if "Thought:" in response:
            thought_text = response.split("Thought:")[-1]
            # Get text until Action or Final Answer
            if "Action:" in thought_text:
                thought_text = thought_text.split("Action:")[0]
            elif "Final Answer:" in thought_text:
                thought_text = thought_text.split("Final Answer:")[0]
            result["thought"] = thought_text.strip()
        
        # Extract Final Answer
        if "Final Answer:" in response:
            result["final_answer"] = response.split("Final Answer:")[-1].strip()
            return result
        
        # Extract Action
        if "Action:" in response:
            action_text = response.split("Action:")[-1]
            action_line = action_text.split("\n")[0].strip()
            result["action"] = action_line
        
        # Extract Action Input
        if "Action Input:" in response:
            input_text = response.split("Action Input:")[-1]
            input_line = input_text.split("\n")[0].strip()
            result["action_input"] = input_line
        
        return result
    
    def _execute_tool(self, tool_name: str, tool_input: str) -> str:
        """Execute a tool and return the result."""
        if tool_name not in self.tools:
            return f"Error: Unknown tool '{tool_name}'. Available: {list(self.tools.keys())}"
        
        try:
            tool = self.tools[tool_name]
            result = tool.invoke(tool_input)
            return str(result)
        except Exception as e:
            return f"Error executing {tool_name}: {str(e)}"
    
    def run(self, question: str, verbose: bool = True) -> str:
        """
        Run the agent on a question.
        
        Args:
            question: The user's question
            verbose: Whether to print intermediate steps
        
        Returns:
            The final answer
        """
        self.history = []
        tool_names = ", ".join(self.tools.keys())
        
        # Build the initial prompt
        prompt = f"""Answer the following question using the available tools.

Available tools:
{self._get_tool_descriptions()}

Use this format:
Thought: [your reasoning]
Action: [tool name from: {tool_names}]
Action Input: [input to the tool]

When you have the final answer:
Thought: I now know the final answer
Final Answer: [your answer]

Question: {question}
Thought:"""
        
        if verbose:
            print(f"\n{'='*50}")
            print(f"❓ Question: {question}")
            print(f"{'='*50}")
        
        # Agent loop
        for iteration in range(self.max_iterations):
            if verbose:
                print(f"\n--- Iteration {iteration + 1} ---")
            
            # Get LLM response
            response = self.llm.invoke(prompt)
            
            # Parse the response
            parsed = self._parse_response(response)
            
            if verbose and parsed["thought"]:
                print(f"💭 Thought: {parsed['thought']}")
            
            # Check for final answer
            if parsed["final_answer"]:
                if verbose:
                    print(f"✅ Final Answer: {parsed['final_answer']}")
                return parsed["final_answer"]
            
            # Execute action if present
            if parsed["action"] and parsed["action_input"]:
                if verbose:
                    print(f"🔧 Action: {parsed['action']}")
                    print(f"📥 Input: {parsed['action_input']}")
                
                observation = self._execute_tool(
                    parsed["action"], 
                    parsed["action_input"]
                )
                
                if verbose:
                    print(f"👁️ Observation: {observation}")
                
                # Add to prompt for next iteration
                prompt += f""" {parsed['thought']}
Action: {parsed['action']}
Action Input: {parsed['action_input']}
Observation: {observation}
Thought:"""
                
                self.history.append({
                    "thought": parsed["thought"],
                    "action": parsed["action"],
                    "action_input": parsed["action_input"],
                    "observation": observation
                })
            else:
                # No action, treat response as attempt at final answer
                if verbose:
                    print(f"⚠️ No action found in response")
                    print(f"   Response: {response[:200]}...")
                return response
        
        return "I reached the maximum number of iterations without finding a complete answer."


print("✅ SimpleReActAgent class defined!")

In [ ]:
# Test our custom agent
print("🧪 Testing SimpleReActAgent:")

custom_agent = SimpleReActAgent(llm, tools, max_iterations=5)

answer = custom_agent.run(
    "What is the square root of 256?",
    verbose=True
)

In [ ]:
# Another test
answer = custom_agent.run(
    "What is DGX Spark's GPU and how many CUDA cores does it have?",
    verbose=True
)

---

## Part 7: Agent with Conversation Memory

Let's add memory so the agent can remember previous conversations.

In [ ]:
# ReAct prompt with chat history
REACT_CHAT_PROMPT = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Previous conversation:
{chat_history}

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

chat_prompt = PromptTemplate.from_template(REACT_CHAT_PROMPT)

# Create memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=False
)

# Create agent with memory
chat_agent = create_react_agent(llm, tools, chat_prompt)

chat_agent_executor = AgentExecutor(
    agent=chat_agent,
    tools=tools,
    memory=memory,
    verbose=True,
    max_iterations=5,
    handle_parsing_errors=True,
)

print("✅ Conversational agent with memory created!")

In [ ]:
# Test conversation memory
print("="*60)
print("🧪 TEST: Conversation with Memory")
print("="*60)

# First question
print("\n📝 Message 1:")
result1 = chat_agent_executor.invoke({"input": "What is 25 * 4?"})
print(f"Answer: {result1['output']}")

# Follow-up that references previous
print("\n📝 Message 2 (references previous):")
result2 = chat_agent_executor.invoke({"input": "Now divide that result by 2"})
print(f"Answer: {result2['output']}")

# Check memory
print("\n💾 Chat History:")
print(memory.load_memory_variables({}))

---

## ⚠️ Common Mistakes

### Mistake 1: No Max Iterations (Infinite Loops)

In [ ]:
print("""
❌ WRONG: No iteration limit

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    # max_iterations not set - could loop forever!
)

✅ RIGHT: Always set max_iterations

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    max_iterations=5,  # Prevents infinite loops
    max_execution_time=60,  # Optional: timeout in seconds
)
""")

### Mistake 2: Not Handling Parsing Errors

In [ ]:
print("""
❌ WRONG: No error handling

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    # handle_parsing_errors not set - crashes on malformed output!
)

✅ RIGHT: Handle parsing errors gracefully

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    handle_parsing_errors=True,  # Recovers from LLM formatting issues
    # Or provide custom handler:
    # handle_parsing_errors="Please format your response correctly."
)
""")

### Mistake 3: Vague Tool Descriptions

In [ ]:
print("""
❌ WRONG: Vague tool description

@tool
def do_math(x: str) -> str:
    \"\"\"Math stuff.\"\"\"  # Too vague! Agent won't know when to use it.
    return eval(x)

✅ RIGHT: Clear, specific description with examples

@tool
def calculate(expression: str) -> str:
    \"\"\"Calculate a mathematical expression.
    
    Use for ANY math: addition, subtraction, multiplication, 
    division, square roots, trigonometry, etc.
    
    Args:
        expression: Math like "2 + 2" or "sqrt(16) * 3"
    
    Examples:
        calculate("15 * 0.2") -> "3"
        calculate("sqrt(144)") -> "12"
    \"\"\"  # Clear when and how to use!
""")

---

## 🎉 Checkpoint

You've learned:
- ✅ The ReAct (Reasoning + Acting) pattern
- ✅ How to create agents with LangChain
- ✅ Building a ReAct agent from scratch
- ✅ Adding conversation memory
- ✅ Common mistakes to avoid

---

## ✋ Try It Yourself

1. Add a new tool to the agent (e.g., a unit converter)
2. Test the agent with a complex multi-step question
3. Modify the SimpleReActAgent to track token usage

---

## 🚀 Challenge (Optional)

Build an agent that can:
1. Search for information about a model
2. Calculate if it fits in DGX Spark memory
3. Recommend the appropriate quantization level

Example query: "Can I run Llama 3.1 405B on DGX Spark? What quantization would I need?"

---

## 📖 Further Reading

- [ReAct Paper (arXiv)](https://arxiv.org/abs/2210.03629)
- [LangChain Agents](https://python.langchain.com/docs/modules/agents/)
- [Agent Types Comparison](https://python.langchain.com/docs/modules/agents/agent_types/)

---

## 🧹 Cleanup

In [ ]:
import gc

# Clear memory
if 'memory' in dir():
    memory.clear()

# GPU cleanup
try:
    import torch
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
except:
    pass

gc.collect()
print("✅ Cleanup complete!")

---

## 🎓 Summary

In this lab, you learned:

| Concept | Key Points |
|---------|------------|
| **ReAct Pattern** | Think → Act → Observe loop for reasoning with tools |
| **Agent Components** | LLM + Prompt + Tools + Executor |
| **Tool Selection** | Agent decides which tool based on descriptions |
| **Memory** | Enables multi-turn conversations |
| **Safety** | Max iterations, error handling, timeouts |

**Next up:** Lab 3.6.3 - LangGraph Workflows with Human-in-the-Loop